In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
import os, sys, shutil

In [2]:
# Set up state variables and parameters 

n_states = 9  # number of state variables
n_params = 15 # number of parameters 

# State variables 
x = [Symbol('P'),
     Symbol('Pa'),
     Symbol('f'),
     Symbol('fa'),
     Symbol('pT'),
     Symbol('T'),
     Symbol('Fg'),
     Symbol('Fb'),
     Symbol('Cl')]

# Parameters
p = [Symbol('k1'),
     Symbol('k2'),
     Symbol('k3'),
     Symbol('k4'),
     Symbol('k5'),
     Symbol('k6'),
     Symbol('k7'),
     Symbol('k8'),
     Symbol('KT'),
     Symbol('KpT'),
     Symbol('Kfa'),
     Symbol('P0'),
     Symbol('f0'),
     Symbol('pT0'),
     Symbol('Fg0')]

In [3]:
# Define time derivatives of state variables  

theta_T = p[8]*x[5]/(1 + p[8]*x[5])
theta_pTfa = p[9]*p[10]*x[3]*x[4]/(1 + p[10]*x[3] + p[9]*p[10]*x[3]*x[4])

r0 = p[0]*x[0]
r1 = p[1]*x[0]*theta_T
r2 = p[2]*x[2]
r3 = p[3]*x[2]*x[5]
r4 = p[4]*x[1]*theta_pTfa
r5 = p[5]*x[6]*x[5]
r6 = p[6]*x[1]*x[7]
r7 = p[7]*x[8]

f0 = -r0 - r1
f1 = r0 + r1 - r6
f2 = -r2 - r3
f3 = r2 + r3
f4 = -r4
f5 = r4 - r5
f6 = -r5
f7 = r5 - r6
f8 = r6 - r7

f = [f0,f1,f2,f3,f4,f5,f6,f7,f8]

In [4]:
# Define Jacobians

# State Jacobian 
# ns x ns
Js = []
for i in range(n_states):
    Js.append([])
    for j in range(n_states):
        Js[i].append(lambdify((x,p),f[i].diff(x[j])))

# Parameter Jacobian
# ns x np
Jp = []
for i in range(n_states):
    Jp.append([])
    for j in range(n_params):
        Jp[i].append(lambdify((x,p),f[i].diff(p[j])))

In [5]:
# Compute State Jacobian
def A(xval,pval):
    Aval = np.zeros((n_states,n_states))
    for i in range(n_states):
        for j in range(n_states):
            Aval[i,j] = Js[i][j](xval,pval)
    return(Aval)
        
# Compute Parameter Jacobian
def B(xval,pval):
    Bval = np.zeros((n_states,n_params))
    for i in range(n_states):
        for j in range(n_params):
            Bval[i,j] = Jp[i][j](xval,pval)
    return(Bval)

In [6]:
# Define time derivatives of state variables 
ffunc = [lambdify((x,p),f[i]) for i in range(n_states)]

# Compute time derivatives of state variables 
def x_prime(xval,pval,Sval):
    dxdt = np.zeros(n_states)
    for i in range(n_states):
        dxdt[i] = ffunc[i](xval,pval)
    return(dxdt)

In [7]:
# Compute time derivatives of sensitivities 

# S = ns x np sensitivity matrix
# A = ns x ns state jacobian matrix
# B = ns x np parameter jacobian matrix 

def S_prime(Sval,pval,xval):
    dSdt = np.matmul(A(xval,pval),Sval) + B(xval,pval)
    return(dSdt)

In [8]:
# Runge-Kutta-Fehlberg Method

# Take step with current step size
def takestep(f,wi,p,args,h):

    k1 = h*f(wi, p, args)
    k2 = h*f(wi + k1/4, p, args)
    k3 = h*f(wi + 3/32*k1 + 9/32*k2, p, args)
    k4 = h*f(wi + 1932/2197*k1 - 7200/2197*k2 + 7296/2197*k3, p, args)
    k5 = h*f(wi + 439/216*k1 - 8*k2 + 3680/513*k3 - 845/4104*k4, p, args)
    k6 = h*f(wi - 8/27*k1 + 2*k2 - 3544/2565*k3 + 1859/4104*k4 - 11/40*k5, p, args)
        
    w_rk4 = wi + 25/216*k1 + 1408/2565*k3 + 2197/4104*k4 - 1/5*k5 # 4th order runge-kutta
    w_rk5 = wi + 16/135*k1 + 6656/12825*k3 + 28561/56430*k4 - 9/50*k5 + 2/55*k6 # 5th order runge-kutta 
    
    return(w_rk4,w_rk5)
    
# Determine stepsize needed for specified accuracy 
def stepsize(f,wi,p,args,h,epsilon):
    
    smallnum = 1e-12
    
    while True: 
        
        rk4,rk5 = takestep(f,wi,p,args,h) # Try with current stepsize  
      
        R = np.max(abs(rk4 - rk5))/h # Check if both step sizes agree within required accuracy 
        delta = 0.84*(epsilon/(R+smallnum))**0.25
        
        
        if R <= epsilon: # Accept stepsize 
            h_next = h*delta 
            break
        else: # If not keep going
            h = h*delta 
      
    return(h,h_next)

In [9]:
# Integrate equations and sensitivities simultaneously 

def RKF45_step(x_prime,S_prime,xi,Si,p,h,epsilon):

    # Calculate state variable required stepsize
    hx,hx_next = stepsize(x_prime,xi,p,Si,h,epsilon)
    
    # Calculated sensitivity required stepsize 
    hS,hS_next = stepsize(S_prime,Si,p,xi,h,epsilon)
    
    # Pick smaller stepsize of two
    h = np.min((hx,hS))
    h_next = np.min((hx_next,hS_next))
    
    # Advance in time 
    x1,extra = takestep(x_prime,xi,p,Si,h)
    S1,extra = takestep(S_prime,Si,p,xi,h)
    
    return(x1,S1,h,h_next)

In [10]:
def solve_ODEs_Sensitivities(x_prime,S_prime,x0,S0,p,tf,epsilon):
    
    time = []
    xt = []
    St = []
    
    t = 0
    time.append(t)
    xt.append(x0)
    St.append(S0)
    
    x = x0
    S = S0
    h = 0.01
    
    while t < tf:
        x,S,h,h_next = RKF45_step(x_prime,S_prime,x,S,p,h,epsilon)
        t = t + h
        h = h_next 
        time.append(t)
        xt.append(x)
        St.append(S)
        
    time = np.array(time)
    xt = np.array(xt)
    St = np.array(St)
    
    return(time,xt,St)

In [11]:
# Test it 

# Rate constants 
k1 = 5e-5
k2 = 3e-2
k3 = 1e-5
k4 = 1e-3
k5 = 1e-2
k6 = 1e-3
k7 = 7e-4
k8 = 4e-5

# Binding constants 
KT = 0.1
KpT = 0.1
Kfa = 0.1

# Initial conditions 
P0 = 40
Pa0 = 0
f0 = 40
fa0 = 0
pT0 = 40
T0 = 0
Fg0 = 40
Fb0 = 0
Cl0 = 0

# Parameters 
p0 = np.array([k1,k2,k3,k4,k5,k6,k7,k8,KT,KpT,Kfa,P0,f0,pT0,Fg0])

# Initial Sensitivities 
S0 = np.zeros((n_states,n_params))
S0[0,11] = 1 # P and P0
S0[2,12] = 1 # f and f0
S0[4,13] = 1 # pT and pT0
S0[6,14] = 1 # Fg and Fg0

# Initial State 
x0 = np.array([P0,Pa0,f0,fa0,pT0,T0,Fg0,Fb0,Cl0])

# Time length 
tf = 5000

# Allowed local truncation error
epsilon = 1e-8

time,xt,St = solve_ODEs_Sensitivities(x_prime,S_prime,x0,S0,p0,tf,epsilon)

In [12]:
def makeplot(x,y,label_x,label_y,title):
    fig = plt.figure(figsize = (8,6))
    plt.plot(x,y)
    plt.xlabel(label_x,fontsize=18)
    plt.ylabel(label_y,fontsize=18)
    plt.title(title,fontsize=20)
    return(fig)

In [13]:
path = '/Users/kieranfitzmaurice/Documents/Research/Models/'
path = path + 'Surface_Mediated/Sensitivity_Analysis'
os.chdir(path)

label_x = 'time [seconds]'

for i in range(n_states):
    statename = str(x[i])
    statepath = path + '/' + statename
    
    if os.path.exists(statepath):
        shutil.rmtree(statepath)
    os.makedirs(statepath)
    os.chdir(statepath)
    
    # plot state variable trajectory
    
    label_y = statename + ' [units]'
    title = statename + '(t)'
    myfig = makeplot(time,xt[:,i],label_x,label_y,title)
    plotname = 't_' + statename + '.png'
    myfig.savefig(plotname,dpi = 300)
    
    for j in range(n_params):
        
        # Plot sensitivity of state variable i to parameter j
        
        paramname = str(p[j])
        label_y = 'd(' + statename + ') / d(' + paramname + ')'
        title = 'Sensitivity of ' + statename + ' to ' + paramname
        myfig = makeplot(time,St[:,i,j],label_x,label_y,title)
        plotname = 'S_' + statename + '_' + paramname + '.png'
        myfig.savefig(plotname,dpi = 300)
        plt.close('all')
        
    os.chdir(path)